In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
import requests
import time

In [3]:
def download_file(url, file_name):
    resp = requests.get(url=url)
    with open(file_name, "wb") as f:
        f.write(resp.content)
        

def get_org_id(code):
    query_url = 'http://www.cninfo.com.cn/new/information/topSearch/query?keyWord={}&maxNum=10'
    resp = requests.post(url=query_url.format(code))
    assert(resp.ok)
    
    results = json.loads(resp.content)
    for r in results:
        if r.get('category') == 'A股':
            return r['orgId']
        
    print('Could not find orgId for {}'.format(code))
    
    
def get_announcements(code):
    query_url = 'http://www.cninfo.com.cn/new/hisAnnouncement/query'
    
    org_id = get_org_id(code)
    payload = {"stock": "{},{}".format(code, org_id), "category": "category_ndbg_szsh;", "tabName": "fulltext"}
    
    resp = requests.post(url=query_url, data=payload)
    assert(resp.ok)
    
    result = json.loads(resp.content)
    
    return result["announcements"]


def download_annual_reports(code):
    download_url = 'http://static.cninfo.com.cn/{}'
    path = '/Users/yuzifeng/GitHub/stock/notebook'   
    
    announcements = get_announcements(code)
    
    if len(announcements) == 0:
        print('Nothing to download')
        return
    
    sec_name = announcements[0]['secName']
    path = os.path.join(path, '{} {}'.format(code, sec_name))
    if not os.path.exists(path):
        print("Create folder {}".format(path))
        os.makedirs(path)
        
    for report in announcements:
        title = report['announcementTitle']
        file_path = os.path.join(path, title + '.pdf')
        if '年度报告' in title and '摘要' not in title and '财务' not in title and not os.path.exists(file_path):
            print('Download {}'.format(file_path))
            download_file(download_url.format(report['adjunctUrl']), file_path)
            time.sleep(1)
        
    print('Download finished.')

            
assert(get_org_id('600858') == 'gssh0600858')

In [4]:
#下载年度报告
download_annual_reports('600582')


Create folder /Users/yuzifeng/GitHub/stock/notebook/600582 天地科技
Download /Users/yuzifeng/GitHub/stock/notebook/600582 天地科技/天地科技2021年年度报告.pdf
Download /Users/yuzifeng/GitHub/stock/notebook/600582 天地科技/天地科技2020年年度报告.pdf
Download /Users/yuzifeng/GitHub/stock/notebook/600582 天地科技/2019年年度报告.pdf
Download /Users/yuzifeng/GitHub/stock/notebook/600582 天地科技/2018年年度报告.pdf
Download /Users/yuzifeng/GitHub/stock/notebook/600582 天地科技/2017年年度报告.pdf
Download /Users/yuzifeng/GitHub/stock/notebook/600582 天地科技/2016年年度报告.pdf
Download /Users/yuzifeng/GitHub/stock/notebook/600582 天地科技/2015年年度报告.pdf
Download /Users/yuzifeng/GitHub/stock/notebook/600582 天地科技/2014年年度报告（更新版）.pdf
Download /Users/yuzifeng/GitHub/stock/notebook/600582 天地科技/2014年年度报告.pdf
Download /Users/yuzifeng/GitHub/stock/notebook/600582 天地科技/2013年年度报告.pdf
Download /Users/yuzifeng/GitHub/stock/notebook/600582 天地科技/2012年年度报告.pdf
Download finished.
